<a href="https://colab.research.google.com/github/salarbalou/Photothermal_Study_CQDs/blob/main/Photothermal_Study_of_CQDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import numpy as np
import plotly.io as pio
pio.templates.default= 'plotly_dark'
from plotly.subplots import make_subplots

In [77]:
colors = ['red', 'cyan']
concentration = '0.75'
dcs = '20', '40', '60', '80', '100'
space = '_'
baseurl = 'https://raw.githubusercontent.com/salarbalou/Photothermal_Study_CQDs/main/'
format = '.txt'

In [126]:
fig = make_subplots()
water_dcs = '20', '40', '60', '80', '100'
HRs  = []
experiments = []
for water_dc in water_dcs:
 data_water = np.loadtxt(baseurl+'water'+space+'1'+space+water_dc+format, delimiter=' , ')[9:]
 tw, Tw = data_water[:,0][0:121], data_water[:,1][0:121]
 fig.add_scatter(x  = tw, y = Tw, name = f'water at PWM_dutycycle {water_dc}')
 fig.update_layout(height  = 800, width = 1000, title =  'Water Heating')
 fig.update_xaxes(nticks = 20, tickfont = dict(size = 18), title = 'time (seconds)', title_font = dict(size = 18))
 fig.update_yaxes(nticks = 20, tickfont = dict(size = 18), title = 'Temperature (˚C)', title_font = dict(size = 18))
for color in colors:
  for dc in dcs:
     data = np.loadtxt(baseurl+color+space+concentration+space+dc+format, delimiter=' , ')[9:]
     experiments.append(f'Color {color}, concentration {concentration}, dc {dc}')
     th = data[:, 0][0:121] #time (s)
     Th = data[:, 1][0:121] #temperature of the solution (C)
     Rh  = np.gradient(Th, th) #gradient of T with respect to time at max which is considered the heating rate
     HR = np.max(Rh) #heating rate
     Teqh = Th[np.where(Rh <= np.min(R))[0]] #equilibrium temperature (C)
     teqh = th[np.where(Rh <= np.min(R))[0]] # time required to reach equilibrium (s)
     fig.add_scatter(x= th, y = Th, name = f'{color}_{concentration}_{dc}', mode = 'markers+lines', marker_color = color)
     fig.update_layout(height  = 800, width = 1000, title =  'CQD Heating')
     fig.update_xaxes(nticks = 25, tickfont = dict(size = 18), title = 'time (seconds)', title_font = dict(size = 18))
     fig.update_yaxes(nticks = 25, tickfont = dict(size = 18), title = 'Temperature (˚C)', title_font = dict(size = 18)) 
     HRs.append(HR)

In [127]:
HRs

[0.7650000000000006,
 2.6799999999999997,
 5.995000000000001,
 8.295000000000002,
 10.074999999999998,
 1.1449999999999996,
 2.549999999999999,
 4.974999999999998,
 7.015000000000001,
 5.484999999999999]

In [129]:
optimized_sample = experiments[np.argmax(HRs)]
print(optimized_sample)

Color red, concentration 0.75, dc 100


In [79]:
#for blue 435nm maxpower=4W laser:
m, b = 7.025524475524473, -66.07575757575763
# dc is arduino board pwm pin duty cycle

In [31]:
# *** Steady State Analysis *** Heating rate **** #turning on laser for 120 seconds to measure the heating rate
t = data[:, 0][0:121] #time (s)
T = data[:, 1][0:121] #temperature of the solution (C)
Tw = np.full(t.size, np.min(T))#temperature of the environment (C) (needs to be measured by another K type thermocouple) assumed to be constant at 22 C
R  = np.gradient(T, t) #gradient of T with respect to time
HR = np.max(R) #heating rate
Teq = T[np.where(R <= np.min(R))[0]] #equilibrium temperature (C)
teq = t[np.where(R <= np.min(R))[0]] # time required to reach equilibrium (s)
print(f'Heated up to equilibrium eemperature of {Teq} in {teq} seconds')

Heated up to equilibrium eemperature of [26.27] in [9.] seconds


In [16]:
HR

5.484999999999999

In [39]:
fig = make_subplots()
fig.add_scatter(x= t, y = T, name = f'{concentration}_{dc}', mode = 'markers+lines', marker_color = color)
fig.update_layout(height  = 800, width = 1000, title =  'CQD Heating')
fig.update_xaxes(nticks = 25, tickfont = dict(size = 18), title = 'time (seconds)', title_font = dict(size = 18))
fig.update_yaxes(nticks = 25, tickfont = dict(size = 18), title = 'Temperature (˚C)', title_font = dict(size = 18))

In [10]:
# *** Steady State Analysis *** Cooling rate **** #turning off laser for 120 seconds to measure the cooling rate
t = data[:, 0] #time (s)
T = data[:, 1] #temperature of the solution (C)
Tw = np.full(t.size, 22)#temperature of the environment (C) (needs to be measured by another K type thermocouple) assumed to be constant at 22 C
C  = np.gradient(T, t) #gradient of T with respect to time
CR = np.max(C) #heating rate
Teq = T[np.min(np.where(C<0.01))] #equilibrium temperature (C)
teq = t[np.min(np.where(C<0.01))] # time required to reach equilibrium (s)
print(f'Cooled down to equilibrium temperature of {Teq} in {teq} seconds')

Cooled down to equilibrium temperature of 108.43 in 34.0 seconds


In [11]:
# *** Photothermal conversion efficiency ***

#Iout = #measured using powermeter: f(CCQD, L_laser) (W)
#S_PCR_tube = #surface area of the solution at which laser beam is passing through:[PCR_tube, paper] (m**2)
#S_paper = #surface are of the paper at which laser beam is covering:[PCR_tube, paper]  (m**2)
#m = rho * V #mass of solution (Kg)

Iin = m*dc + b #input power of the laser: f(dc) (W)
h = 1000 #heat transfor coefficient of solution with its surrounding (W/(m**2)*C)

rho  = 1000 #density of solution (kg/m**3)
V = 1e-8 #volume of the solution (m**3)
Cp = 4182 #heat capacity of water (J/Kg*C)
#eta = ((R*(Cp*m)+np.sum((h*S*(T - Tw))))/(Iin - Iout)
#print(f'Photothermal Conversion efficiency of your CQD is {eta}')

TypeError: ignored

In [ ]:
# *** what we need to know of each CQD***
#print(f' {Color} CQD reached Teq of {Teq} at {Iout} mW in {teq} seconds, eta: {eta}')

In [ ]:
# ***Modeling the heat diffusion in PCR-Tube and paper***
nx = 2
x = np.arange(nx)
nt = np.int(teq) #temperature rise is evaluated in {t.size} seconds
Tplot = np.full(T.size, np.min(Tw))

In [ ]:
fig = make_subplots()
fig.add_scatter(x = x, y = Tplot, name = '0 s')
for i in range (nt):
  Tplot[0] = T[i]
  Tplot[-1] = np.min(Tw)
  fig.add_scatter(x = np.arange(nx), y = Tplot, name = f'{i+1} s')
fig.update_layout(height = 800, width = 1100)
fig.show()